In [9]:
import psycopg2
import pandas as pd
from psycopg2.extras import RealDictCursor

# Database connection parameters from docker-compose.yml
CONNECTION = 'host=postgres port=5432 dbname=mydb user=user password=password'

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor(cursor_factory=RealDictCursor)


In [10]:
cur.execute(f"SELECT * FROM stations")
result = cur.fetchall()
conn.commit()

In [11]:
result

[RealDictRow([('station_id', 'VS1721'),
              ('station_name', 'Stavanger (SW) - Risavika')]),
 RealDictRow([('station_id', 'VS1596'),
              ('station_name', 'Stavanger (S) - DSD')]),
 RealDictRow([('station_id', 'VS1595'),
              ('station_name', 'Stavanger (N) - Tjuvholmen')]),
 RealDictRow([('station_id', 'VS1722'),
              ('station_name', 'Stavanger (NW) - Mekjarvik')])]

In [12]:
query = """SELECT count(*) FROM stations"""
cur.execute(query)
result = cur.fetchone()
conn.commit()
result

RealDictRow([('count', 4)])

In [13]:

query = """SELECT last(wind_speed, timestamp) FROM wind_measurements"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
result

[RealDictRow([('last', 0.4)])]

In [35]:
query = """SELECT * FROM wind_measurements"""
query = """SELECT timestamp, wind_speed FROM wind_measurements WHERE timestamp BETWEEN '2025-03-09T07:10:33.387Z' AND '2025-03-09T23:40:33.387Z' ORDER BY 1"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
pd.DataFrame(result).sort_values('timestamp')

,timestamp,wind_speed
0,2025-03-09 07:45:24,0.0
1,2025-03-09 07:45:25,0.1
2,2025-03-09 07:45:26,0.1
3,2025-03-09 07:45:27,0.0
4,2025-03-09 07:45:28,0.0
...,...,...
2395,2025-03-09 19:46:47,0.7
2396,2025-03-09 19:46:48,0.9
2397,2025-03-09 19:46:49,0.8
2398,2025-03-09 19:46:50,0.7


In [23]:
query = """SELECT bucket as time, station_id, avg_speed as wind_speed, max_speed, min_speed FROM wind_measurements_15min WHERE bucket BETWEEN '2025-03-09T07:24:45.98' AND '2025-03-09T07:54:45.98' ORDER BY 1"""
cur.execute(query)
result = cur.fetchall()
conn.commit()
pd.DataFrame(result)

""
